# Загрузка необходимых библиотек

In [2]:
import requests
from fake_useragent import UserAgent
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup
from IPython.display import clear_output
from fake_useragent import UserAgent
import json
from collections import Counter

import time
import datetime
import pandas as pd
import pickle


from tqdm import tqdm
from joblib import Parallel, delayed

from multiprocessing.pool import ThreadPool


import swifter
import pymorphy3

from selenium import webdriver

# На вход подается большой файл с леммами, на выход - обновленный файл с леммами

In [8]:
def tass_lenta(before, limit=200):
    mainpage = 'http://tass.ru/api/news/lenta?limit='+str(limit)+'&before='+str(before)
    response = requests.get(mainpage)
    dic = response.json()
    cur_news = [ ]
    df = dic['articles']
    for item in df:
        try:
            new = { }
            new['title'] = item['title']
            new['category'] = item['section']['title']
            new['href'] = item['url']
            new['date'] = datetime.datetime.fromtimestamp(int(item['time'])).strftime('%Y-%m-%d %H:%M:%S')
            new['uci_time'] = int(item['time'])
            cur_news.append(new)
        except:
            print('Error! \n',item, '\n')
    return cur_news

Выбор текущей даты и даты последнего обновления файла с новостями

In [3]:
# загружаем общий файл с лемматизированными новостями, вытащим из него последнюю дату
date_list = []
with open("tass/tass_lemmas/lemma_tass_v2.pickle", "rb") as file:
    while True:
        try:
            item=pickle.load(file)['date']
            date_list.append(item)
        except EOFError:
            break

In [6]:
last_date = max(date_list)
print(f'Последняя записанная дата: {last_date}')

last_date_timest = datetime.datetime.strptime(last_date, '%Y-%m-%d %H:%M:%S').timestamp()
print(f'Временная метка последней записанной даты: {last_date_timest}\n')

current_date = datetime.datetime.now()
print(f'Сегодняшняя дата: {current_date}')

print(f'Временная метка сегодняшней даты: {current_date.timestamp()}')

Последняя записанная дата: 2024-07-02 18:10:17
Временная метка последней записанной даты: 1719933017.0

Сегодняшняя дата: 2024-07-12 13:51:04.625278
Временная метка сегодняшней даты: 1720781464.625278


In [7]:
limit = int(current_date.timestamp())
before = int(last_date_timest)

'http://tass.ru/api/news/lenta?limit='+str(limit)+'&before='+str(before)

'http://tass.ru/api/news/lenta?limit=1720781464&before=1719933017'

In [8]:
# создание json-файла, в который будут записываться необработанные новости
for i in range(100000):
    clear_output(wait=True)
    current_news = tass_lenta(limit)
    limit = current_news[-1]['uci_time']

    with open('news_update.json', 'a') as outfile:
        json.dump(current_news, outfile)
        print(i, current_news[-1]['date'])
        
    if limit < pd.to_datetime(last_date).timestamp():    
        break 

46 2024-07-02 20:17:16


In [10]:
with open('news_update.json') as json_file:
    tass_news = json.loads(json_file.read().replace("][", ","))
print(f'Количество новых новостей: {len(tass_news)}')

Количество новых новостей: 9400


Подсчет количества новых новостей по месяцам

In [11]:
test_d = [pd.to_datetime(i['date'][:10]) for i in tass_news]
dts = pd.Series(test_d).sort_values()

dts.value_counts().resample('ME').sum()

2024-07-31    9400
Freq: ME, Name: count, dtype: int64

In [12]:
# категории, которые встретились в пуле новостей
categories_add_json = set([i['category'] for i in tass_news])

In [13]:
select_cat = [
            #     '"Дальневосточный гектар"',
             'Brexit',
             'RIW 2015',
             'Аналитика',
             'Арктика',
             'Армия и ОПК',
             'Банк историй успеха',
             'Без политики',
             'Безопасная жизнь',
             'Безопасность',
             'Биографии и справки',
             'Борьба с коррупцией',
             'Будущее',
             'В стране',
             'ВЭФ-2015',
             'ВЭФ-2016',
             'ВЭФ-2017',
            #  'Взрыв в Бангкоке',
            #  'Взрыв в детском лагере в Подмосковье',
             'Воздушно-космическая оборона',
             'Вокруг Олимпиады в Сочи',
             'Вооруженные силы',
             'Вопрос-ответ',
            #  'Вторая мировая война',
             'Выбор редакции',
             'Выборы депутатов Госдумы',
             'Выборы-2016',
            #  'Выставка Аero India-2015',
             'Гайдаровский форум - 2016',
            #  'ДТП с автобусом и поездом под Владимиром',
            #  'ЕГЭ',
             'Евразийская интеграция',
             'Закон о банкротстве физлиц',
            #  'Здоровье',
             'Инвестфорум "Сочи-2016"',
             'ИноТомск',
             'Интервью Москва',
            #  'КХЛ',
             'Колебания курса рубля',
             'Комментарии',
             'Конференции с онлайн-трансляцией',
             'Коррупционные скандалы',
             'Космос',
             'Кризис на Украине',
             'Криминал',
             'Кубок конфедераций-2017',
            #  'Культура',
             'Лесные пожары в России',
            #  'Летние виды спорта',
            #  'Личное время',
            #  'Люди и вещи',
             'Малый бизнес',
             'Малый бизнес. Интервью',
             'Малый бизнес. Репортажи',
             'Международная панорама',
            #  'Миграционный кризис в Европе',
             'Мнения',
             'Москва',
             'Московская область',
             'Наводнения в России',
             'Наука',
             'Научные новости',
             'Научные технологии',
            #  'Наши люди',
            #  'Нештатный запуск "Прогресса М-27М"',
             'Новости Архангельской области',
             'Новости агентства',
             'Новости партнеров',
             'Новости форума',
             'Образование',
             'Общество',
            #  'Олимпийские и Паралимпийские игры 2016 года',
             'Опросы',
            #  'Освоение Арктики',
             'Отставки и назначения',
             'ПМЭФ-2015',
             'ПМЭФ-2016',
             'ПМЭФ-2017',
            #  'Память о Великой Отечественной войне',
             'Политика',
             'Происшествия',
            #  'Протесты в Ереване',
             'Прямая линия с президентом РФ',
             'Развитие МСП в России',
             'Развитие моногородов',
            #  'Расследование крушения российского самолета в Египте',
            #  'Религия',
             'Саммит АТЭС - 2017',
            #  'Сборные по футболу',
             'Северо-Запад',
             'Сибирь',
            #  'Ситуация в Крыму',
            #  'Ситуация вокруг Греции',
            #  'Ситуация вокруг ядерной программы Ирана',
            #  'Сотрудничество в рамках СНГ',
             'Социальное страхование',
            #  'Спорт',
            #  'Стихийные бедствия',
            #  'ТАСС-ДОСЬЕ / Биографии',
             'ТЭК',
             'Технологии',
             'Транспорт',
            #  'Туризм в России',
            #  'Туризм и отдых',
             'Форум в Давосе',
            #  'Футбол',
            #  'Хоккей',
             'Цифровая экономика',
            #  'ЧП с малайзийским самолетом Airbus',
            #  'Чемпионат WorldSkills',
             'ШОС',
            #  'Экология',
             'Экономика и бизнес',
                '"Технопром – 2019"',
                #  '300-летие Перми',
                #  '75-летие Победы',
                 'Cаммит G20',
                #  'VII Санкт-Петербургский международный культурный форум',
                #  'XXXII летние Олимпийские игры',
                 'Автомобили',
                 'АПК',
                #  'Автоспорт',
                #  'Акции протеста в Иране',
                #  'Акции протеста во Франции',
                #  'Акции протеста на Кубе',
                #  'Атака на гостиничный комплекс в Найроби',
                 'БРИКС',
                #  'Берлинский кинофестиваль',
                #  'Беспорядки в Бразилии',
                #  'Блокада Ленинграда',
                #  'Бокс',
                 'Борьба с изменениями климата',
                 'В мире',
                 'ВЭФ-2018',
                 'ВЭФ-2019',
                #  'Вакцинация от коронавируса в России',
                #  'Взрыв газа в Новосибирске',
                #  'Взрыв газа в жилом доме в Ельце',
                #  'Визит Нэнси Пелоси на Тайвань',
                 'Внешняя политика',
                 'Внутренняя политика',
                #  'Военная операция США в Сирии',
                 'Военная операция на Украине',
                 'Восточный экономический форум',
                #  'Всероссийские учения энергетиков в Дагестане',
                #  'Встреча Путина и Байдена в Женеве',
                #  'Вторая мировая война',
                #  'Выборы в ДНР и ЛНР',
                #  'Выборы президента США - 2020',
                #  'Дела в отношении экс-министра Михаила Абызова',
                #  'Дело Виктора Бута',
                #  'Дело Дональда Трампа',
                #  'Дело Кирилла Вышинского',
                #  'Дело Кирилла Серебренникова',
                #  'Дело Эдварда Сноудена',
                #  'Дело в отношении Кокорина и Мамаева',
                #  'Дело в отношении Сергея Фургала',
                #  'Дело летчика Ярошенко',
                #  'Дело полковника Захарченко',
                #  'День Победы',
                 'Динамика ключевой ставки Центробанка',
                #  'Допинг в спорте',
                 'Единый день голосования',
                #  'Жилье и городская среда',
                #  'Задержание Саакашвили в Грузии',
                #  'Задержание финансового директора Huawei',
                #  'Здоровье',
                #  'Здравоохранение',
                #  'Землетрясение и цунами в Индонезии',
                #  'Землетрясения в Турции и Сирии',
                #  'Зимние виды спорта',
                 'ИННОПРОМ-2018',
                #  'Инцидент с британским эсминцем в Черном море',
                #  'Инциденты с подозрительными посылками в США',
                #  'Искусство',
                #  'Каннский кинофестиваль',
                #  'Каннский фестиваль',
                #  'Катастрофа Boeing-737 под Тегераном',
                 'Кибербезопасность',
                #  'Кино',
                #  'Крушение самолета Bek Air в Казахстане',
                 'Кузбасс',
                #  'Культура',
                #  'Лесные пожары в Калифорнии',
                #  'Литература',
                #  'Личное время',
                #  'Люди и вещи',
                 'Макроэкономика',
                 'Малое и среднее предпринимательство',
                 'Медицина',
                 'Международный опыт',
                #  'Московская конференция по безопасности',
                #  'Московский кинофестиваль ',
                #  'Музыка',
                #  'НХЛ',
                 'Национальные проекты',
                 'Нацпроект "Наука и университеты"',
                 'Нацпроект "Экология"',
                #  'Наши люди',
                 'Недвижимость',
                 'Нижегородская область',
                 'Нобелевская премия - 2019',
                 'Новая система обращения с ТКО',
                 'Новости Урала',
                 'Новостные разделы',
                 'Новый год и Рождество - 2021',
                 'Оборонно-промышленный комплекс',
                #  'Обострение конфликта на киргизско-таджикской границе',
                #  'Обострение на границе Армении и Азербайджана',
                #  'Обрушение перекрытий в университете в Петербурге',
                #  'Обрушение эскалатора на станции метро в Риме',
                #  'Олимпиада-2018',
                #  'Олимпиада-2022 в Пекине',
                 'Опросы общественного мнения',
                #  'Освоение Арктики',
                 'Отношения России и НАТО',
                #  'Отставка президента Перу',
                 'ПМЭФ ',
                 'ПМЭФ-2018',
                 'ПМЭФ-2022',
                #  'Палестино-израильский конфликт',
                #  'Пандемия COVID-19',
                #  'Пенсионный закон',
                 'Первые лица',
                 'Первые лица бизнеса',
                 'Первые лица российских регионов',
                #  'Переговоры Путина и Байдена',
                 'Переговоры России и США по гарантиям безопасности',
                #  'Подкасты',
                #  'Подписание договоров о вхождении в РФ новых субъектов',
                #  'Политический кризис в Абхазии',
                #  'Политический кризис в Венесуэле',
                #  'Политический кризис в Молдавии',
                #  'Православная Пасха',
                #  'Президентские выборы в США',
                #  'Президентские выборы во Франции',
                #  'Провозглашение независимости Каталонии',
                #  'Промежуточные выборы в Конгресс США',
                 'Промышленность',
                #  'Протесты в Казахстане',
                #  'Противодействие терроризму в России',
                #  'Процедура импичмента Трампа',
                #  'Расследование крушения Boeing в Донецкой области',
                #  'Религия',
                 'Репортаж',
                 'Российский инвестиционный форум - 2018',
                 'Российский инвестиционный форум - 2019',
                 'Саммит "нормандской четверки" в Париже',
                 'Саммит Россия – Африка',
                 'Санкции в отношении России',
                 'Северный Кавказ',
                 'Ситуация в зоне нагорнокарабахского конфликта',
                 'Ситуация вокруг Промсвязьбанка',
                 'Ситуация вокруг экстренной посадки самолета Ryanair в Минске',
                 'Ситуация вокруг ядерной программы Ирана',
                 'Ситуация на Корейском полуострове',
                #  'Ситуация с мигрантами на границе Польши и Белоруссии',
                 'Ситуация с поставками энергоносителей из РФ в Европу',
                #  'Смена власти в Афганистане',
                 'Соглашение о продлении ДСНВ',
                #  'Создание Автокефальной поместной православной церкви на Украине',
                 'Социальная защита',
                #  'Спорт',
                #  'Стихийные бедствия',
                #  'Столкновения в Судане',
                #  'Стрельба в США',
                #  'Театр',
                #  'Туризм в России',
                #  'Туризм и отдых',
                #  'Умер Леонид Куравлев',
                #  'Умер Николай Добронравов',
                #  'Фигурное катание',
                #  'Фрагменты новых книг',
                #  'Футбол',
                #  'Хоккей',
                #  'ЧМ-2018 по футболу',
                #  'Чемпионат WorldSkills',
                #  'Экология',
                'Электроэнергетика',
                'Эскалация в Косове',
                'Авиационные системы',
                'Визит Владимира Путина в Китай',
                'Инаугурация президента РФ',
                'Обострение ситуации между Израилем и Ираном',
]

In [14]:
# новые категории
categories_add_json - set(select_cat)

{'Дело Ассанжа и WikiLeaks',
 'Культура',
 'Президентские выборы в США',
 'Протесты в Кении',
 'Спорт'}

# 1. Загрузка новостей

In [13]:
def page_content(url):
    time.sleep(0.63)
    response = requests.get(url, timeout=20)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    
    try_1 = soup.find_all("div", {"class": "NewsHeader_lead__6Z9If"})
    if len(try_1) != 0:
        snippet = try_1[0].text
        all_par = soup.find("article").find_all("p")
        text = ' '.join([i.text for i in all_par])
    
    elif len(soup.find_all("div", {"class": "news-header"})) != 0:
        snippet =soup.find_all("div", {"class": "news-header"})[0].text
        text = soup.find("div", {'class':'text-content'}).text
        
    elif len(soup.find_all("h1", {"class": re.compile(r'tass_pkg_title.+')}))!=0:
        snippet = soup.find_all("h1", {"class": re.compile(r'tass_pkg_title.+')})[0].text.strip()
        all_par = soup.find("article").find_all("p")
        text = ' '.join([i.text for i in all_par])

    else:
        snippet = soup.find_all("div", {"class": re.compile(r'.+Header_wrapper.+')})[0].text
        all_par = soup.find("article").find_all("p")
        text = ' '.join([i.text for i in all_par])
        
        
    return text, snippet

In [14]:
def get_tass_news(item_from_vect, pt):
    url = "http://tass.ru" + item_from_vect['href']
    try:
        text, snippet = page_content(url)
        item_from_vect['text'] = text.strip()
        item_from_vect['snippet'] = snippet.strip()
        
        on_tdf = pd.DataFrame([item_from_vect])
        on_tdf[['date', 'category', 'title', 'text', 'href']].to_csv(f'tass/tass_texts{pt}.csv',
                                                              mode='a', header=False, index=None)

    except: 
        on_tdf = pd.DataFrame([item_from_vect])
        on_tdf[['date', 'category', 'title', 'href']].to_csv(f'tass/skip_urls_v1.csv',
                                                              mode='a', header=False, index=None)
        print(url)
        return{ }

In [15]:
def Separator(vect, part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    out = [[str(i), vect[vec_parts[i]:vec_parts[i+1]]] for i in range(part)]
    return(out)

def Map(vect, parser_function):
    for i, item in enumerate(vect[1]):
        clear_output(wait=True)
        print(f'{vect[0]}:{i}/{len(vect[1])}')
        parser_function(item, vect[0])

def MRDownloader(what, parts, parser_function):
    separatorlist = Separator(what, parts)
    def Mp(what):
        return Map(what, parser_function=parser_function)
    pool = ThreadPool(parts)
    l = pool.map(Mp, separatorlist)

In [16]:
separatorlist = Separator(tass_news, 15)

len(separatorlist), len(separatorlist[0]), len(separatorlist[0][1])

MRDownloader(tass_news, 15, parser_function=get_tass_news)

6:626/627


Пропущенные ссылки

In [17]:
skip_ref = pd.read_csv('tass/skip_urls_v1.csv', header=None)
skip_ref.columns = ['date', 'category', 'title', 'href']
skip_ref = skip_ref.drop_duplicates(subset=['title', 'href'])

In [18]:
exsist_ref = []
df_tab = []
for i in range (15):
    df_temp = pd.read_csv(f'tass/tass_texts{i}.csv', header=None) 
    df_tab.append(df_temp)
    exsist_ref.append(df_temp[4].values)
    
exsist_ref = sum([list(i) for i in exsist_ref],[])

print(f'Количество обработанных новостей: {len(exsist_ref)}')   
tass_news = [i for i in tass_news if i['href'] not in exsist_ref]
print(f'Количество пропущенных новостей: {len(tass_news)}')

Количество обработанных новостей: 9257
Количество пропущенных новостей: 143


# 2. Обработка пропущенных новостей

на вход подается csv-файл с пропущенными ссылками (полученный ранее skip_urls_v1.csv), на выход - tass_texts16.csv

In [3]:
cService = webdriver.ChromeService(executable_path='C:\\Users\\Дяна\\OneDrive\\Рабочий стол\\lab\\parse_unparsed\\chromedriver.exe')
driver = webdriver.Chrome(service = cService)


def content_driver(url):  
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    if len(soup.find_all('h1', {'class': 'explainer__title'})) != 0:        
        title = soup.find('h1', {'class': 'explainer__title-wrap'}).text
        text = soup.find('div', {'class': re.compile('text-content.+')}).find_all('p')
        text = ' '.join([i.text for i in text]) 
        text = text.replace('\xa0', '')
    

    elif len(soup.find_all('h1', {'class': 'news-header__title'})) != 0:
        title = soup.find_all('h1', {'class': 'news-header__title'})[0].text     
        text = soup.find_all("div", {"class": "text-block"})[0].find_all("p")
        text = ' '.join([i.text for i in text])
    
    
    else:
        title = soup.find_all('h1', {'class': 'article__title-wrap'})[0].text
        text = soup.find_all('div', {'class': re.compile('text-content.+')})[0] 
        text = ' '.join([i.text for i in text])
        text = text.replace('\xa0', ' ')
        text = text.replace('\n', ' ')
    
    
    return text, title

In [4]:
def for_missing(skipped_urls_csv):
    file = pd.read_csv(skipped_urls_csv, names = ['date', 'category', 'title', 'href', 'text'], header=None)
    
    for idx, row in file.iterrows():
        try:
            url = 'http://tass.ru' + str(file['href'][idx])     
            file['text'][idx] = str(content_driver(url)[0])
            pd.DataFrame([file.loc[idx, :]])[['date', 'category', 'title', 'text', 'href']].to_csv('tass/tass_texts15.csv', mode = 'a', 
                                                                                                   index=None, header=None)
        except:
            pd.DataFrame([file.loc[idx, :]])[['date', 'category', 'title', 'href']].to_csv('tass/still_skipped.csv', mode = 'a', 
                                                                                           index=None, header=None)

In [5]:
for_missing('tass/skip_urls_v1.csv')

C:\Users\Дяна\AppData\Local\Temp\ipykernel_5000\3704882431.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['text'][idx] = str(content_driver(url)[0])


In [6]:
print(f'Дополнительно обработано новостей: {pd.read_csv("tass/tass_texts15.csv", header=None).shape[0]}')
print(f'Итоговое число пропущенных новостей: {pd.read_csv("tass/still_skipped.csv", header=None).shape[0]}')

Дополнительно обработано новостей: 131
Итоговое число пропущенных новостей: 10


# 3. Предобработка текстов

In [19]:
m = pymorphy3.MorphAnalyzer()

def pymorphy_lemma(text='The text must be passed to the function as a string!'):
    text = re.sub(r'[^а-яёА-Я]+', ' ', text)
    tokens = text.lower().split()   
    lemmas = []
    
    for word in tokens:
        try:
            lemma = m.parse(word)[0].normal_form
            lemmas.append(lemma)
        except:
            continue
    return ', '.join(lemmas)

In [20]:
path_tass = 'tass/'
for fn in range(16):
    files  = path_tass + f'tass_texts{fn}.csv'
    df = pd.read_csv(files, header=None)
    df.columns = ['date', 'category', 'title', 'text', 'href']
    df = df.drop_duplicates(subset=['title', 'text'])

# проверка, что значения являются str
    df['test'] = df['title'].swifter.apply(lambda x: type(x))
    df = df[df.test==str]
    df['test'] = df['text'].swifter.apply(lambda x: type(x))
    df = df[df.test==str]
    df = df.drop('test', axis=1)

    with open(path_tass + "tass_lemmas/lemma_tass_update.pickle", 'ab') as file: 
        for i in tqdm(df.index):

            line = df.loc[i].to_dict()

            if 'text' in line:
                line["text"] = pymorphy_lemma(text=line["text"])
            else:
                line["text"] = ['']

            if 'title' in line:
                line["title"] = pymorphy_lemma(text=line["title"]) 
            else:
                line['title'] = ['']

            pickle.dump(line, file)
    file.close()

Pandas Apply:   0%|          | 0/622 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/622 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 622/622 [00:59<00:00, 10.43it/s]


Pandas Apply:   0%|          | 0/610 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/610 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/614 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 614/614 [00:35<00:00, 17.43it/s]


Pandas Apply:   0%|          | 0/611 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/611 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 611/611 [00:37<00:00, 16.14it/s]


Присоединяем update к старому файлу с леммами

In [88]:
path_fp = 'tass/tass_lemmas/'
final_pickle = open(path_fp + "lemma_tass_v2.pickle", 'ab')

path_tass = 'tass/tass_lemmas/'
with open(path_tass + "lemma_tass_update.pickle", 'rb') as file:
    while True:
        try:
            item = pickle.load(file)
            pickle.dump(item, final_pickle)
        except EOFError:
            break

file.close()
final_pickle.close()

In [90]:
lemma_final=[]
with open("tass/tass_lemmas/lemma_tass_v2.pickle", "rb") as file:
    while True:
        try:
            item=pickle.load(file)['date']
            lemma_final.append(item)
        except EOFError:
            break
            
print(f'Длина обновленного файла: {len(lemma_final)}')

Длина псл файла: 66397
